# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [4]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """
    url: str
    title: str
    body: str
    links: List[str]
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
print(ed.get_contents())

Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of pr

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [7]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/
https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/
https://edwarddonner.com/2024/08/06/outsmart/
https://e

In [8]:
def get_links(url):
    website = Website(url)
    completion = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = completion.choices[0].message.content
    return json.loads(result)

In [9]:
get_links("https://anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://anthropic.com/careers'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [10]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [11]:
print(get_all_details("https://anthropic.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'}, {'type': 'api page', 'url': 'https://anthropic.com/api'}, {'type': 'pricing page', 'url': 'https://anthropic.com/pricing'}]}
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Claude
Overview
Team
Enterprise
API
Pricing
Research
Company
Careers
News
AI
research
and
products
that put safety at the frontier
Claude.ai
Meet Claude 3.5 Sonnet
Claude 3.5 Sonnet, our most intelligent AI model, is now available.
Talk to Claude
API
Build with Claude
Start using Claude to drive efficiency and create new revenue streams.
Learn more
Announcements
Introducing computer use, a new Claude 3.5 Sonnet, and Claude 3.5 Haiku
Oct 22, 2024
Model u

In [12]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [13]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncate if more than 20,000 characters
    return user_prompt

In [14]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [15]:
create_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'news page', 'url': 'https://anthropic.com/news'}]}


# Anthropic Brochure

## Overview
Anthropic is a pioneering AI safety and research company headquartered in San Francisco. Committed to building reliable, interpretable, and steerable AI systems, Anthropic's mission is to ensure that transformative AI technologies contribute positively to society. Our interdisciplinary team combines expertise from machine learning, policy, physics, and product management to create innovative AI solutions.

## Our Product: Claude
At the heart of Anthropic's offerings lies **Claude**, a powerful AI model designed to enhance productivity and collaboration across industries. The latest versions, **Claude 3.5 Sonnet** and **Claude 3.5 Haiku**, empower teams to leverage AI for various applications, from content creation to complex data analysis.

- **Collaboration**: Claude serves as a virtual teammate, streamlining daily tasks and fostering a culture of innovation.
- **Efficiency**: By utilizing Claude, teams can significantly reduce the time spent on documents, coding, and brainstorming sessions.
  
## Company Culture
### Our Values
1. **Here for the Mission**: Our focus is to build AI systems that ensure societal benefits, aiming to tackle emerging challenges responsibly.
2. **Unusually High Trust**: We uphold an environment of trust, encouraging open communication and collaborative decision-making.
3. **One Big Team**: Collaboration is essential; we operate as a united team despite having various specialized units.
4. **Do the Simple Thing That Works**: We favor practical solutions and empirical evidence to guide our decisions.

### Team Dynamics
Our diverse team comprises researchers, engineers, policy experts, and operational leaders who all contribute to our collective goals. We embrace a culture that recognizes the importance of collaboration, innovation, and the pursuit of practical problem-solving.

## Customers
Anthropic partners with businesses, nonprofits, and government entities. Our clientele ranges from small startups to large enterprises that leverage Claude to drive efficiencies and create new revenue streams. Notable testimonials highlight the drastic improvements in project throughput and creativity attributed to Claude's capabilities.

## Careers at Anthropic
Join us in making AI safe for everyone! At Anthropic, we offer a range of competitive benefits and opportunities for professional growth, including:

- **Health & Wellness**: Comprehensive insurance plans, parental leave, mental health support, and wellness stipends.
- **Professional Development**: Educational stipends and flexible work arrangements.
- **Work Environment**: We support both remote work transitions and team collaboration in our office space.

### Hiring Process
Our hiring process includes:
1. **Resume Submission**: Initial review of qualifications.
2. **Exploratory Chat**: Discuss your interests and experiences with our team.
3. **Skills Assessment**: Based on the role you apply for.
4. **Interviews with Team Members**: Assessing both technical and cultural fit.

We welcome diverse backgrounds and are dedicated to providing equal opportunities, including visa sponsorship.

## Why Choose Anthropic?
Anthropic exists at the intersection of technology and ethics. By joining our team, leveraging our products, or investing in our mission, you contribute to a future where AI is safe, reliable, and interpretable, ultimately enriching society as a whole.

For further inquiries or to explore career opportunities or partnerships, please visit our website or contact us directly.

---

*Connect with us: [Twitter](https://twitter.com/anthropic) | [LinkedIn](https://www.linkedin.com/company/anthropic) | [YouTube](https://www.youtube.com/c/Anthropic)*

*© 2024 Anthropic PBC. All rights reserved.*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [16]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [17]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'}, {'type': 'api page', 'url': 'https://anthropic.com/api'}, {'type': 'pricing page', 'url': 'https://anthropic.com/pricing'}, {'type': 'news page', 'url': 'https://anthropic.com/news'}]}


# Anthropic: Pioneering Safe and Beneficial AI

### About Us
Anthropic is an AI safety and research company headquartered in San Francisco. Our mission is to create reliable, interpretable, and steerable AI systems that prioritize safety. We believe AI has the potential to significantly impact the world and are dedicated to building systems that can be trusted to enhance human flourishing.

### Our Vision
At Anthropic, we view AI safety as a systematic science. Our research spans various modalities and safety techniques, which we integrate into our products, providing a collaborative platform for civil society, government, academia, and industry to promote safety across the AI landscape.

### Meet Claude
Claude is our advanced AI model, designed for both enterprises and individual users. Claude empowers users to innovate efficiently by providing insights, drafting documents, generating code, and improving team collaboration. It is engineered to complement human expertise and enhance productivity within organizations.

### Company Culture
- **Mission-Driven**: We exist to ensure that transformative AI helps society flourish. Our rapid progress in AI technology demands that we proactively address the challenges it presents.
- **High Trust**: We foster an environment of mutual respect, honesty, and collaboration. Our team operates under the belief that open communication and emotional maturity enhance our decision-making.
- **Interdisciplinary Collaboration**: We are a diverse team of researchers, engineers, policy experts, and operators, working collectively to push the envelope on AI safety.
- **Simplicity**: We value practical, effective solutions over overly complex ones, embracing an empirical approach to our research and development.

### Careers at Anthropic
We’re growing and seeking talented individuals who share our passion for safe AI. Our team members come from various backgrounds, including physics, machine learning, public policy, and business. 

#### What We Offer:
- **Health & Wellness**: Comprehensive health, dental, and vision insurance, paid parental leave, flexible paid time off, and generous mental health support.
- **Compensation**: Competitive salary and equity packages, retirement plans, and life protection plans.
- **Additional Benefits**: Options for wellness stipends, education stipends, relocation support, and daily meals in the office.
  
Join us in shaping the future of AI by exploring our open roles and engaging in a dynamic interview process that emphasizes your strengths and potential impact.

### Collaborate with Us
Anthropic thrives on partnerships and collaboration to enhance our product offerings and research insights. We encourage inquiries from potential clients and organizations interested in leveraging Claude's capabilities across various domains, including marketing, engineering, sales, and research.

### Stay Connected
To learn more about Anthropic, our products, and how we can work together to foster a safe AI future, visit our website or follow us on social media.

- [Twitter](https://twitter.com/Anthropic)
- [LinkedIn](https://linkedin.com/company/anthropic)
- [YouTube](https://youtube.com/c/Anthropic)

---

**Explore the possibilities with Anthropic!**

In [18]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Company Brochure

## About Us
Hugging Face is at the forefront of the AI revolution, dedicated to building a collaborative machine learning community. Our mission is to democratize machine learning through open-source solutions and collective contributions. With a thriving platform where over 50,000 organizations engage, we serve as a central hub for models, datasets, and applications, empowering enthusiasts and professionals alike.

### Core Features
- **Models**: Access to over 400,000 models, fostering creativity and innovation within the machine learning ecosystem.
- **Datasets**: A repository of over 100,000 datasets available for various tasks, making data accessibility easier than ever.
- **Spaces**: Innovative applications where various models are deployed for practical use like image generation and text processing.

## Company Culture
At Hugging Face, we believe in the strength of community-driven development. Our team comprises 221 talented members, all united by a shared passion for advancing machine learning. We encourage collaboration, inclusivity, and creative problem-solving to push the boundaries of what's possible in AI.

- **Collaboration Over Competition**: Join a culture that prioritizes teamwork and open communication.
- **Learning Mindset**: We foster continuous learning and professional development, urging every member to explore and innovate.
- **Commitment to Open Source**: Our foundational belief is in the open-source model, promoting accessibility and transparency.

## Customers
Our platform is embraced by diverse customers across sectors, from enterprises like Amazon Web Services, Google, and Microsoft to innovative startups. Here are a few highlights:
- **AI at Meta**
- **Grammarly**
- **Intel**

Over 336 models have been developed for customer use, demonstrating our commitment to enhancing various applications.

## Career Opportunities
We're always on the lookout for passionate individuals who want to join our mission of democratizing machine learning. Hugging Face offers numerous career paths across various departments—including engineering, community management, and research. 

**Current Open Positions**
- Software Engineers
- Data Scientists
- Community Managers

Join us in creating the future of AI. Explore our [Careers Page](https://huggingface.co/jobs) for the latest openings!

## Join the Community
If you're enthusiastic about machine learning and AI, we invite you to become a part of our journey. Whether you're a developer looking to contribute, a researcher seeking high-quality datasets, or an enterprise in need of robust AI solutions, Hugging Face is the platform for you!

### Stay Connected
Follow us on our social media channels and become part of a vibrant community pushing the boundaries of AI.

- [Twitter](https://twitter.com/huggingface)
- [GitHub](https://github.com/huggingface)
- [LinkedIn](https://www.linkedin.com/company/huggingface)

---

For additional information, please visit our website: [Hugging Face](https://huggingface.co) 

Together, let's build the future of AI!

## Business Applications

In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

In terms of techniques, this is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

In terms of business applications - generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.